In [1]:
using DifferentiableStateSpaceModels, LinearAlgebra, Optim, Turing, Zygote
using DifferentiableStateSpaceModels.Examples
using Turing: @addlogprob!
Turing.setadbackend(:zygote)

# Create models from modules and then solve
model_rbc = @include_example_module(Examples.rbc_observables)

# Generate artificial data for estimation
p_f = (ρ=0.2, δ=0.02, σ=0.01, Ω_1=0.01) # Fixed parameters
p_d = (α=0.5, β=0.95) # Pseudo-true values
sol = generate_perturbation(model_rbc, p_d, p_f, Val(1))
sol_second = generate_perturbation(model_rbc, p_d, p_f, Val(2))

T = 20
ϵ = [randn(model_rbc.n_ϵ) for _ in 1:T]
x0 = zeros(model_rbc.n_x)
fake_z = solve(sol, x0, (0, T), DifferentiableStateSpaceModels.LTI(); noise = ϵ).z
fake_z_second = solve(sol_second, x0, (0, T), DifferentiableStateSpaceModels.QTI(); noise = ϵ).z

21-element Array{Array{Float64,1},1}:
 [7.824904812715083e-5, 0.0]
 [0.0010623135708196184, -7.824904812715083e-5]
 [-0.00528965775606637, 0.008973074029422561]
 [-0.010350250463222034, -0.04878671372604625]
 [-0.009577254064857573, -0.09969332132204582]
 [-0.0016549078042811772, -0.09642148047235256]
 [-0.01037035353167762, -0.02325633439122632]
 [-0.006059793449768964, -0.09794524838161296]
 [-0.004287244017558761, -0.06392230261437896]
 [-0.007091036173358491, -0.045057009282125415]
 [0.0014596239812387825, -0.06943587583600722]
 [-0.0024092502832767215, 0.0074392973936722165]
 [-0.01297502619875524, -0.022407214474767713]
 [-0.022877650137168172, -0.12182302431171685]
 [-0.02216604801470862, -0.22037368586767184]
 [-0.029414439199145798, -0.22128635950460115]
 [-0.015576645172797643, -0.28797839539986786]
 [-0.02115712262490475, -0.1657082087262903]
 [-0.03490028504834272, -0.20788783891765156]
 [-0.022706926724819117, -0.3373388822061155]
 [-0.013448165982173056, -0.23501403229102

In [4]:
## Estimation example: first-order, marginal likelihood approach

# Turing model definition
@model function rbc_kalman(z, m, p_f, cache)
    α ~ Uniform(0.2, 0.8)
    β ~ Uniform(0.5, 0.99)
    p_d = (α = α, β = β)
    sol = generate_perturbation(m, p_d, p_f, Val(1); cache)
    if !(sol.retcode == :Success)
        @addlogprob! -Inf
        return
    end
    @addlogprob! solve(sol, sol.x_ergodic, (0, length(z)); observables = z).logpdf
end

c = SolverCache(model_rbc, Val(1), p_d)
turing_model = rbc_kalman(fake_z, model_rbc, p_f, c)
n_samples = 1000
n_adapts = 100
δ = 0.65
chain = sample(turing_model, NUTS(n_adapts, δ), n_samples; progress = true)

┌ Info: Found initial step size
│   ϵ = 0.025
└ @ Turing.Inference C:\Users\wupei\.julia\packages\Turing\y0DW3\src\inference\hmc.jl:188
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:57


Chains MCMC chain (1000×14×1 Array{Float64,3}):

Iterations        = 101:1:1100
Number of chains  = 1
Samples per chain = 1000
Wall duration     = 120.4 seconds
Compute duration  = 120.4 seconds
parameters        = α, β
internals         = lp, n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, max_hamiltonian_energy_error, tree_depth, numerical_error, step_size, nom_step_size

Summary Statistics
  parameters      mean       std   naive_se      mcse        ess      rhat   e ⋯
      Symbol   Float64   Float64    Float64   Float64    Float64   Float64     ⋯

           α    0.4912    0.0291     0.0009    0.0016   322.2872    0.9991     ⋯
           β    0.9503    0.0116     0.0004    0.0007   253.8297    0.9990     ⋯
                                                                1 column omitted

Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float64 

          

In [11]:
## Estimation example: first-order, joint likelihood approach

# Turing model definition
@model function rbc_joint(z, m, p_f, cache, x0 = zeros(m.n_x))
    α ~ Uniform(0.2, 0.8)
    β ~ Uniform(0.5, 0.99)
    p_d = (α = α, β = β)
    T = length(z)
    ϵ_draw ~ MvNormal(T, 1.0)
    ϵ = map(i -> ϵ_draw[((i-1)*m.n_ϵ+1):(i*m.n_ϵ)], 1:T)
    # println(p_d)
    sol = generate_perturbation(m, p_d, p_f, Val(1); cache)
    if !(sol.retcode == :Success)
        @addlogprob! -Inf
        return
    end
    @addlogprob! solve(sol, x0, (0, T); noise = ϵ, observables = z).logpdf
end

c = SolverCache(model_rbc, Val(1), p_d)
turing_model = rbc_joint(fake_z, model_rbc, p_f, c)
n_samples = 1000
n_adapts = 100
δ = 0.65
max_depth = 5 # A lower max_depth will lead to higher autocorrelation of samples, but faster. The time complexity is approximately 2^max_depth
chain = sample(turing_model, NUTS(n_adapts, δ; max_depth), n_samples; progress = true)

┌ Info: Found initial step size
│   ϵ = 0.00625
└ @ Turing.Inference C:\Users\wupei\.julia\packages\Turing\y0DW3\src\inference\hmc.jl:188
Sampling: 100%|█████████████████████████████████████████| Time: 0:01:49


Chains MCMC chain (1000×35×1 Array{Float64,3}):

Iterations        = 101:1:1100
Number of chains  = 1
Samples per chain = 1000
Wall duration     = 113.0 seconds
Compute duration  = 113.0 seconds
parameters        = α, ϵ_draw[5], ϵ_draw[8], ϵ_draw[12], ϵ_draw[16], ϵ_draw[3], ϵ_draw[2], ϵ_draw[17], ϵ_draw[20], ϵ_draw[10], ϵ_draw[19], ϵ_draw[18], ϵ_draw[9], ϵ_draw[1], ϵ_draw[15], ϵ_draw[13], β, ϵ_draw[6], ϵ_draw[7], ϵ_draw[11], ϵ_draw[14], ϵ_draw[4], ϵ_draw[21]
internals         = lp, n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, max_hamiltonian_energy_error, tree_depth, numerical_error, step_size, nom_step_size

Summary Statistics
  parameters      mean       std   naive_se      mcse       ess      rhat   es ⋯
      Symbol   Float64   Float64    Float64   Float64   Float64   Float64      ⋯

           α    0.4449    0.0282     0.0009    0.0063   13.2219    1.0550      ⋯
           β    0.9553    0.0063     0.0002    0.0008   53.3171    1.

In [2]:
## Estimation example: second-order, joint likelihood approach

# Turing model definition
@model function rbc_second(z, m, p_f, cache, x0 = zeros(m.n_x))
    α ~ Uniform(0.2, 0.8)
    β ~ Uniform(0.5, 0.99)
    p_d = (α = α, β = β)
    T = length(z)
    ϵ_draw ~ MvNormal(T, 1.0)
    ϵ = map(i -> ϵ_draw[((i-1)*m.n_ϵ+1):(i*m.n_ϵ)], 1:T)
    sol = generate_perturbation(m, p_d, p_f, Val(2); cache)
    if !(sol.retcode == :Success)
        @addlogprob! -Inf
        return
    end
    @addlogprob! solve(sol, x0, (0, T); noise = ϵ, observables = z).logpdf
end

c = SolverCache(model_rbc, Val(2), p_d)
turing_model = rbc_second(fake_z_second, model_rbc, p_f, c)
n_samples = 1000
n_adapts = 100
δ = 0.65
max_depth = 5
chain = sample(turing_model, NUTS(n_adapts, δ; max_depth), n_samples; progress = true)

┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\wupei\.julia\packages\AdvancedHMC\HQHnm\src\hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\wupei\.julia\packages\AdvancedHMC\HQHnm\src\hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\wupei\.julia\packages\AdvancedHMC\HQHnm\src\hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\wupei\.julia\packages\AdvancedHMC\HQHnm\src\hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = 

Chains MCMC chain (1000×35×1 Array{Float64,3}):

Iterations        = 101:1:1100
Number of chains  = 1
Samples per chain = 1000
Wall duration     = 314.21 seconds
Compute duration  = 314.21 seconds
parameters        = α, ϵ_draw[5], ϵ_draw[8], ϵ_draw[12], ϵ_draw[16], ϵ_draw[3], ϵ_draw[2], ϵ_draw[17], ϵ_draw[20], ϵ_draw[10], ϵ_draw[19], ϵ_draw[18], ϵ_draw[9], ϵ_draw[1], ϵ_draw[15], ϵ_draw[13], β, ϵ_draw[6], ϵ_draw[7], ϵ_draw[11], ϵ_draw[14], ϵ_draw[4], ϵ_draw[21]
internals         = lp, n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, max_hamiltonian_energy_error, tree_depth, numerical_error, step_size, nom_step_size

Summary Statistics
  parameters      mean       std   naive_se      mcse        ess      rhat   e ⋯
      Symbol   Float64   Float64    Float64   Float64    Float64   Float64     ⋯

           α    0.5169    0.0222     0.0007    0.0042    27.7736    1.0342     ⋯
           β    0.9487    0.0084     0.0003    0.0009   115.3721   